# <center> ML-7. Оптимизация гиперпараметров модели

In [70]:
#Импорты

#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные модели
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import hp, fmin, STATUS_OK, tpe, Trials
import optuna

%matplotlib inline
plt.style.use('seaborn-v0_8')

In [11]:
data = pd.read_csv('data/_train_sem09.csv')

data

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Проверяем, есть ли пропуски в любом из столбцов
has_missing = data.isnull().any().any()

if has_missing:
    print("В датафрейме есть пропуски.")
else:
    print("В датафрейме нет пропусков.")

В датафрейме нет пропусков.


In [13]:
#Смотрим распреление значений целевого признака
data['Activity'].value_counts(normalize=True)

Activity
1    0.542255
0    0.457745
Name: proportion, dtype: float64

Разбиение почти ровное, но стратификация, думаю, не повредит.

In [14]:
# Разделение данных на выборки

X = data.drop(['Activity'], axis=1)
y = data['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

In [15]:
#Построение изначальных базовых моделей
print('Логистическая регрессия:')
lr_base = linear_model.LogisticRegression(random_state=42, max_iter=1000)

lr_base.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(lr_base.score(X_test, y_test)))
y_test_pred = lr_base.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


print('-'*50)
print('Случайный лес:')
rf_base = ensemble.RandomForestClassifier(random_state=42)

rf_base.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(rf_base.score(X_test, y_test)))
y_test_pred = rf_base.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Логистическая регрессия:
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.78
--------------------------------------------------
Случайный лес:
accuracy на тестовом наборе: 0.81
f1_score на тестовом наборе: 0.83


Базовые митрики есть. Приступаем к оптимизации гиперпараметров.

## <center> Grid Search

In [16]:
#Линейная регрессия
param_grid_lr = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

grid_search_lr = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=1000 #количество итераций на сходимость
    ), 
    param_grid=param_grid_lr, 
    cv=5, 
    n_jobs = -1
)  

%time grid_search_lr.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_lr.score(X_test, y_test)))
y_test_pred = grid_search_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_lr.best_params_))

c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
70 fits failed out of a total of 280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.p

CPU times: total: 2.23 s
Wall time: 3min 29s
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}


In [19]:
#Случайный лес
param_grid_rf = {
                'n_estimators': list(range(50, 200, 25)), #Количество деревьев
                'min_samples_leaf': list(range(2, 10, 2)), # Количество листьев
                'max_depth': list(range(10, 30, 5)) # Максимальная высота дерева
               }

grid_search_rf = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(
        random_state=42, #генератор случайных чисел
    ), 
    param_grid=param_grid_rf, 
    cv=5, 
    n_jobs = -1
)  

%time grid_search_rf.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_rf.score(X_test, y_test)))
y_test_pred = grid_search_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_rf.best_params_))

CPU times: total: 5.19 s
Wall time: 1min 55s
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'max_depth': 15, 'min_samples_leaf': 2, 'n_estimators': 75}


## <center> Random Search

In [20]:
#Линейная регрессия

random_search_lr = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42),
    param_distributions=param_grid_lr,
    cv=5,
    n_iter=20,
    n_jobs=-1
)

%time random_search_lr.fit(X_train, y_train) 
y_train_pred = random_search_lr.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_lr.score(X_test, y_test)))
y_test_pred = random_search_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_lr.best_params_))

c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
35 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
11 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.p

CPU times: total: 953 ms
Wall time: 18.1 s
f1_score на обучающем наборе: 0.80
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.1}


In [21]:
#Случайный лес
random_search_rf = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_distributions=param_grid_rf,
    cv=5,
    n_iter=20,
    n_jobs=-1
)

%time random_search_rf.fit(X_train, y_train) 
y_train_pred = random_search_rf.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_rf.score(X_test, y_test)))
y_test_pred = random_search_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_rf.best_params_))

CPU times: total: 2.59 s
Wall time: 14.6 s
f1_score на обучающем наборе: 0.99
accuracy на тестовом наборе: 0.83
f1_score на тестовом наборе: 0.84
Наилучшие значения гиперпараметров: {'n_estimators': 150, 'min_samples_leaf': 2, 'max_depth': 20}


## <center> Hyperopt

In [87]:
# зададим пространство поиска гиперпараметров
# Параметры для линейной регрессии
space_lr_1={
    'penalty': hp.choice('penalty', ['l2']),
    'solver': hp.choice('solver', ['lbfgs', 'sag']),
    'C': hp.quniform('C', 0.01, 1, 0.1)
}
space_lr_2={
    'penalty': hp.choice('penalty', ['l1', 'l2']),
    'solver': hp.choice('solver', ['saga', 'liblinear']),
    'C': hp.quniform('C', 0.01, 1, 0.1)
}

#Параметры для случайного леса
space_rf={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

random_state=42
max_iter=1000

In [88]:
# Логистическая регрессия
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': str(params['penalty']), 
              'solver': str(params['solver']), 
              'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, max_iter=max_iter, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return {'loss': -score, 'params': params, 'status': STATUS_OK}



# Случайный лес
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [89]:
%%time
#Подбор параметров для логистической регрессии

# Для первой подборки алгоритмов оптимизации
trials = Trials() # используется для логирования результатов

best_lr_1=fmin(hyperopt_lr, # наша функция 
          space=space_lr_1, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_lr_1))
params_lr_1 = trials.best_trial['result']['params']

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 20/20 [09:08<00:00, 27.41s/trial, best loss: -0.7786349481373416]
Наилучшие значения гиперпараметров {'C': 0.1, 'penalty': 0, 'solver': 0}
CPU times: total: 5min 7s
Wall time: 9min 8s


In [90]:
%%time
# Для второй подборки алгоритмов оптимизации
trials = Trials() # используется для логирования результатов

best_lr_2=fmin(hyperopt_lr, # наша функция 
          space=space_lr_2, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_lr_2))
params_lr_2 = trials.best_trial['result']['params']

 35%|███▌      | 7/20 [02:12<02:35, 11.93s/trial, best loss: -0.7781558796810103]

c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 60%|██████    | 12/20 [06:14<03:36, 27.07s/trial, best loss: -0.7781558796810103]

c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 95%|█████████▌| 19/20 [09:54<00:16, 16.26s/trial, best loss: -0.7816671635398847]

c:\Users\telis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|██████████| 20/20 [12:10<00:00, 36.55s/trial, best loss: -0.7816671635398847]
Наилучшие значения гиперпараметров {'C': 0.1, 'penalty': 0, 'solver': 1}
CPU times: total: 5min 54s
Wall time: 12min 10s


In [91]:
%%time
#Подбор параметров для случайного леса

trials = Trials() # используется для логирования результатов

best_rf=fmin(hyperopt_rf, # наша функция 
          space=space_rf, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров для случайного леса {}".format(best_rf))

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 20/20 [01:38<00:00,  4.90s/trial, best loss: -0.81625044714172]  
Наилучшие значения гиперпараметров для случайного леса {'max_depth': 18.0, 'min_samples_leaf': 2.0, 'n_estimators': 103.0}
CPU times: total: 47.7 s
Wall time: 1min 38s


In [93]:
# рассчитаем точность для тестовой выборки
#Логистическая регрессия 1
model = linear_model.LogisticRegression(
    random_state=random_state,
    max_iter=max_iter,
    penalty=str(params_lr_1['penalty']),
    solver=str(params_lr_1['solver']),
    C=float(params_lr_1['C'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


print('-'*50)
# Логистическая регрессия 2
model = linear_model.LogisticRegression(
    random_state=random_state,
    max_iter=max_iter,
    penalty=str(params_lr_2['penalty']),
    solver=str(params_lr_2['solver']),
    C=float(params_lr_2['C'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


print('-'*100)
# Случайный лес
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best_rf['n_estimators']),
    max_depth=int(best_rf['max_depth']),
    min_samples_leaf=int(best_rf['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.85
accuracy на тестовом наборе: 0.77
f1_score на тестовом наборе: 0.79
--------------------------------------------------
f1_score на обучающем наборе: 0.80
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.81
----------------------------------------------------------------------------------------------------
f1_score на обучающем наборе: 0.99
accuracy на тестовом наборе: 0.81
f1_score на тестовом наборе: 0.82
